<a href="https://colab.research.google.com/github/vaishnabala/Lending_case_study/blob/master/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e7/sample_submission.csv
/kaggle/input/playground-series-s3e7/train.csv
/kaggle/input/playground-series-s3e7/test.csv
/kaggle/input/reservation-cancellation-prediction/train__dataset.csv
/kaggle/input/reservation-cancellation-prediction/test___dataset.csv


In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e7/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e7/test.csv')
train_add = pd.read_csv('/kaggle/input/reservation-cancellation-prediction/train__dataset.csv')
#displaying few rows
train.head()

,id,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0
1,1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0
2,2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0
3,3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0
4,4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42100 entries, 0 to 42099
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    42100 non-null  int64  
 1   no_of_adults                          42100 non-null  int64  
 2   no_of_children                        42100 non-null  int64  
 3   no_of_weekend_nights                  42100 non-null  int64  
 4   no_of_week_nights                     42100 non-null  int64  
 5   type_of_meal_plan                     42100 non-null  int64  
 6   required_car_parking_space            42100 non-null  int64  
 7   room_type_reserved                    42100 non-null  int64  
 8   lead_time                             42100 non-null  int64  
 9   arrival_year                          42100 non-null  int64  
 10  arrival_month                         42100 non-null  int64  
 11  arrival_date   

In [ ]:
#checking for null values
print("null values in train : ",sum(train.isnull().sum()))
print("null values in test : ",sum(test.isnull().sum()))
print("null values in train_add : ",sum(train_add.isnull().sum()))

null values in train :  0
null values in test :  0
null values in train_add :  0


In [ ]:
cat_col = []
for i, col in enumerate(test.columns):
    if len(test[col].value_counts().index) <=10:
        cat_col.append(col)
cat_col = [
    'required_car_parking_space',
    'market_segment_type',
    'room_type_reserved',
    'repeated_guest',
    'type_of_meal_plan'
]

#cat_col.append('arrival_date')
#cat_col.append('no_of_previous_bookings_not_canceled')

def cat_conv(dataset,col_list):
    for col in col_list:
        dataset[col]= dataset[col].astype('category')

cat_conv(train,cat_col)
cat_conv(test,cat_col)
cat_conv(train_add,cat_col)

#saving id for submission
test_id = test['id']
train.drop(['id'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)
train_add = train_add[train.columns]
test = test[[col for col in train.columns if col!='booking_status']]

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42100 entries, 0 to 42099
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   no_of_adults                          42100 non-null  int64   
 1   no_of_children                        42100 non-null  int64   
 2   no_of_weekend_nights                  42100 non-null  int64   
 3   no_of_week_nights                     42100 non-null  int64   
 4   type_of_meal_plan                     42100 non-null  category
 5   required_car_parking_space            42100 non-null  category
 6   room_type_reserved                    42100 non-null  category
 7   lead_time                             42100 non-null  int64   
 8   arrival_year                          42100 non-null  int64   
 9   arrival_month                         42100 non-null  int64   
 10  arrival_date                          42100 non-null  int64   
 11  ma

In [ ]:
#importing ml libraries and classes
import catboost as cat
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.target_encoder import TargetEncoder
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras import Sequential, Model, Input


In [ ]:
#splitting dataset into training and CV
X_train, X_cv, y_train, y_cv = train_test_split(train.drop(['booking_status'],axis=1),train[['booking_status']], test_size=0.2, shuffle=True,random_state=1)
#print(X_train.shape, X_cv.shape)

#instantiate encoder
train_encoder = CatBoostEncoder()
X_train = train_encoder.fit_transform(X_train,y_train)
X_cv = train_encoder.transform(X_cv)
X_test = train_encoder.transform(test)
#instantiate additional dataset encoder
train_add_encoder = CatBoostEncoder()
X_add = train_add_encoder.fit_transform(train_add.drop(['booking_status'],axis=1),train_add[['booking_status']])
y_add = train_add[['booking_status']]

X_train = pd.concat([X_train,X_add],axis=0)
y_train = pd.concat([y_train,y_add],axis=0)

In [ ]:
#ann
def dnn(learning_rate=0.001):
    inputs = Input(shape=(17))
    X = inputs
    for i in [512,256,128,64]:
        X = Dense(units=i)(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
    X = Dense(units=1)(X)
    X = BatchNormalization()(X)
    outputs = Activation('sigmoid')(X)
    model = Model(inputs =inputs, outputs=outputs)
    print(model.summary())
    model.compile(loss='BinaryCrossentropy',optimizer = tf.keras.optimizers.Adam(tf.keras.optimizers.schedules.ExponentialDecay(0.001,3200,0.9)), metrics=['AUC'])
    history = model.fit(X_train,y_train, epochs=300, batch_size=64, validation_data=(X_cv,y_cv))#, callbacks=[tf.keras.callbacks.LearningRateScheduler(lambda epoch : 1e-06*10**(epoch/20))])

    return model, history

In [ ]:
#clear model instances
tf.keras.backend.clear_session()

#model
model, history = dnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17)]              0         
                                                                 
 dense (Dense)               (None, 512)               9216      
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                             

In [ ]:
pred = (model.predict(X_test)>0.5).flatten().astype('int')

878/878 [==============================] - 2s 2ms/step


In [ ]:
submission = pd.DataFrame({'id':test_id,'booking_status':pred})
submission.to_csv('submission.csv',index=False)

import matplotlib.pyplot as plt
plt.figure(figsize=(15,9))
plt.plot(np.arange(1,1001),history.history['loss'],label='train_loss')
plt.plot(np.arange(1,1001),history.history['val_loss'],label='val_loss')
plt.legend()
plt.show()

import matplotlib.pyplot as plt
plt.figure(figsize=(15,9))
plt.plot(np.arange(1,1001),history.history['auc'],label='train_auc')
plt.plot(np.arange(1,1001),history.history['val_auc'],label='val_auc')
plt.legend()
plt.show()